In [1]:
import geopandas as gpd, folium, branca
from folium.plugins import Search

In [2]:
folium.__version__

'0.6.0+23.ga044e82'

Let's get some JSON data from the web - both a point layer and a polygon GeoJson dataset with some population data.

In [3]:
states = gpd.read_file(r"https://rawcdn.githack.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json", driver='GeoJSON')

cities = gpd.read_file(r"https://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_50m_populated_places_simple.geojson", driver='GeoJSON')

And take a look at what our data looks like:

In [4]:
states.describe()

,density
count,52.000000
mean,402.504404
std,1395.100812
min,1.264000
25%,53.440000
50%,100.335000
75%,234.050000
max,10065.000000


Look how far the minimum and maximum values for the density are from the top and bottom quartile breakpoints! We have some outliers in our data that are well outside the meat of most of the distribution. Let's look into this to find the culprits within the sample.

In [5]:
states_sorted = states.sort_values(by='density', ascending=False)

states_sorted.head(5).append(states_sorted.tail(5))[['name','density']]

,name,density
8,District of Columbia,10065.000
30,New Jersey,1189.000
51,Puerto Rico,1082.000
39,Rhode Island,1006.000
21,Massachusetts,840.200
31,New Mexico,17.160
34,North Dakota,9.916
26,Montana,6.858
50,Wyoming,5.851
1,Alaska,1.264


Looks like Washington D.C. and Alaska were the culprits on each end of the range. Washington was more dense than the next most dense state, New Jersey, than the least dense state, Alaska was from Wyoming, however. Washington D.C. has a has a relatively small land area for the amount of people that live there, so it makes sense that it's pretty dense. And Alaska has a lot of land area, but not much of it is habitable for humans.
<br><br>
However, we're looking at all of the states in the US to look at things on a more regional level. That high figure at the top of our range for Washington D.C. will really hinder the ability for us to differentiate between the other states, so let's account for that in the min and max values for our color scale, by getting the quantile values close to the end of the range. Anything higher or lower than those values will just fall into the 'highest' and 'lowest' bins for coloring.

In [6]:
min, max = states['density'].quantile([0.05,0.95]).apply(lambda x: round(x, 2))

mean = round(states['density'].mean(),2)


print(f"Min: {min}", f"Max: {max}", f"Mean: {mean}", sep="\n\n")

Min: 8.54

Max: 1040.2

Mean: 402.5


This looks better. Our min and max values for the colorscale are much closer to the mean value now. Let's run with these values, and make a colorscale. I'm just going to use a sequential light-to-dark color palette from the [ColorBrewer](http://colorbrewer2.org/#type=sequential&scheme=Purples&n=5).

In [7]:
colormap = branca.colormap.LinearColormap(
    colors=['#f2f0f7','#cbc9e2','#9e9ac8','#756bb1','#54278f'],
    index=states['density'].quantile([0.2,0.4,0.6,0.8]),
    vmin=min,
    vmax=max
)

colormap.caption="Population Density in the United States"

colormap


<svg height="50" width="500"><line x1="0" y1="0" x2="0" y2="20" style="stroke:#f2f0f7;stroke-width:3;" /><line x1="1" y1="0" x2="1" y2="20" style="stroke:#f2f0f7;stroke-width:3;" /><line x1="2" y1="0" x2="2" y2="20" style="stroke:#f2f0f7;stroke-width:3;" /><line x1="3" y1="0" x2="3" y2="20" style="stroke:#f2f0f7;stroke-width:3;" /><line x1="4" y1="0" x2="4" y2="20" style="stroke:#f2f0f7;stroke-width:3;" /><line x1="5" y1="0" x2="5" y2="20" style="stroke:#f2f0f7;stroke-width:3;" /><line x1="6" y1="0" x2="6" y2="20" style="stroke:#f2f0f7;stroke-width:3;" /><line x1="7" y1="0" x2="7" y2="20" style="stroke:#f2f0f7;stroke-width:3;" /><line x1="8" y1="0" x2="8" y2="20" style="stroke:#f2f0f7;stroke-width:3;" /><line x1="9" y1="0" x2="9" y2="20" style="stroke:#f2f0f7;stroke-width:3;" /><line x1="10" y1="0" x2="10" y2="20" style="stroke:#f2f0f7;stroke-width:3;" /><line x1="11" y1="0" x2="11" y2="20" style="stroke:#f2f0f7;stroke-width:3;" /><line x1="12" y1="0" x2="12" y2="20" style="stroke:#f2f0f7;stroke-width:3;" /><line x1="13" y1="0" x2="13" y2="20" style="stroke:#f2f0f7;stroke-width:3;" /><line x1="14" y1="0" x2="14" y2="20" style="stroke:#f2f0f7;stroke-width:3;" /><line x1="15" y1="0" x2="15" y2="20" style="stroke:#f2f0f7;stroke-width:3;" /><line x1="16" y1="0" x2="16" y2="20" style="stroke:#f2f0f7;stroke-width:3;" /><line x1="17" y1="0" x2="17" y2="20" style="stroke:#f0eef6;stroke-width:3;" /><line x1="18" y1="0" x2="18" y2="20" style="stroke:#eeecf5;stroke-width:3;" /><line x1="19" y1="0" x2="19" y2="20" style="stroke:#eceaf4;stroke-width:3;" /><line x1="20" y1="0" x2="20" y2="20" style="stroke:#eae8f3;stroke-width:3;" /><line x1="21" y1="0" x2="21" y2="20" style="stroke:#e8e6f2;stroke-width:3;" /><line x1="22" y1="0" x2="22" y2="20" style="stroke:#e6e4f1;stroke-width:3;" /><line x1="23" y1="0" x2="23" y2="20" style="stroke:#e4e2f0;stroke-width:3;" /><line x1="24" y1="0" x2="24" y2="20" style="stroke:#e2e0ee;stroke-width:3;" /><line x1="25" y1="0" x2="25" y2="20" style="stroke:#e0deed;stroke-width:3;" /><line x1="26" y1="0" x2="26" y2="20" style="stroke:#dedcec;stroke-width:3;" /><line x1="27" y1="0" x2="27" y2="20" style="stroke:#dcdaeb;stroke-width:3;" /><line x1="28" y1="0" x2="28" y2="20" style="stroke:#dad8ea;stroke-width:3;" /><line x1="29" y1="0" x2="29" y2="20" style="stroke:#d8d6e9;stroke-width:3;" /><line x1="30" y1="0" x2="30" y2="20" style="stroke:#d6d4e8;stroke-width:3;" /><line x1="31" y1="0" x2="31" y2="20" style="stroke:#d4d2e7;stroke-width:3;" /><line x1="32" y1="0" x2="32" y2="20" style="stroke:#d2d0e6;stroke-width:3;" /><line x1="33" y1="0" x2="33" y2="20" style="stroke:#d0cee5;stroke-width:3;" /><line x1="34" y1="0" x2="34" y2="20" style="stroke:#cecce4;stroke-width:3;" /><line x1="35" y1="0" x2="35" y2="20" style="stroke:#cccae3;stroke-width:3;" /><line x1="36" y1="0" x2="36" y2="20" style="stroke:#cac8e2;stroke-width:3;" /><line x1="37" y1="0" x2="37" y2="20" style="stroke:#c9c7e1;stroke-width:3;" /><line x1="38" y1="0" x2="38" y2="20" style="stroke:#c7c5e0;stroke-width:3;" /><line x1="39" y1="0" x2="39" y2="20" style="stroke:#c6c4df;stroke-width:3;" /><line x1="40" y1="0" x2="40" y2="20" style="stroke:#c5c3df;stroke-width:3;" /><line x1="41" y1="0" x2="41" y2="20" style="stroke:#c4c1de;stroke-width:3;" /><line x1="42" y1="0" x2="42" y2="20" style="stroke:#c2c0dd;stroke-width:3;" /><line x1="43" y1="0" x2="43" y2="20" style="stroke:#c1bedc;stroke-width:3;" /><line x1="44" y1="0" x2="44" y2="20" style="stroke:#c0bddc;stroke-width:3;" /><line x1="45" y1="0" x2="45" y2="20" style="stroke:#bebcdb;stroke-width:3;" /><line x1="46" y1="0" x2="46" y2="20" style="stroke:#bdbada;stroke-width:3;" /><line x1="47" y1="0" x2="47" y2="20" style="stroke:#bcb9d9;stroke-width:3;" /><line x1="48" y1="0" x2="48" y2="20" style="stroke:#bab8d9;stroke-width:3;" /><line x1="49" y1="0" x2="49" y2="20" style="stroke:#b9b6d8;stroke-width:3;" /><line x1="50" y1="0" x2="50" y2="20" style="stroke:#b8b5d7;stroke-width:3;" /><line x1="51"

Let's narrow down these cities to United states cities, by using GeoPandas' spatial join functionality between two GeoDataFrame objects, using the Point 'within' Polygon functionality.

In [8]:
us_cities = gpd.sjoin(cities, states, how='inner', op='within')
pop_ranked_cities = us_cities.sort_values(by='pop_max', ascending=False)[['nameascii','pop_max', 'geometry'
                                                               ]].iloc[:20]

Ok, now we have a new GeoDataFrame with our top 20 populated cities. Let's see the top 5.

In [9]:
pop_ranked_cities.head(5)

,nameascii,pop_max,geometry
1224,New York,19040000,POINT (-73.98196278740681 40.75192492259464)
1222,Los Angeles,12500000,POINT (-118.1819263699404 33.99192410876543)
1186,Chicago,8990000,POINT (-87.75200083270931 41.83193651927843)
1184,Miami,5585000,POINT (-80.22605193945003 25.78955655502153)
1076,Philadelphia,5492000,POINT (-75.17194183200792 40.00191902252647)


Alright, let's build a map!

In [ ]:
# Instantiate our folium Map.
m = folium.Map(location=[38,-97], zoom_start=4)

# Create some folium GeoJson objects from our GeoDataFrames.
stategeo = folium.GeoJson(states,
                          name='US States',
                          style_function=lambda x: {'fillColor': colormap(x['properties']['density']), 'color': 'black',
                                                    'weight':2, 'fillOpacity':0.5},
                          tooltip=folium.GeoJsonTooltip(fields=['name', 'density'], 
                                            aliases=['State', 'Density'], 
                                            localize=True)
                         ).add_to(m)

citygeo = folium.GeoJson(pop_ranked_cities,
                         name='US Cities',
                         tooltip=folium.GeoJsonTooltip(fields=['nameascii','pop_max'],
                                                       aliases=['','Population Max'],
                                                       localize=True)
                        ).add_to(m)

# Add some Search boxes to the map that reference the GeoDataFrames with some different parameters passed to the 
# arguments.
statesearch = Search(layer=stategeo, 
                     geom_type='Polygon', 
                     placeholder="Search for a US State", 
                     collapsed=False, 
                     search_label='name',
                     weight=3
                    ).add_to(m)

citysearch = Search(layer=citygeo, 
                    geom_type='Point', 
                    placeholder="Search for a US City", 
                    collapsed=True, 
                    search_label='nameascii'
                   ).add_to(m)

# Add a LayerControl.
folium.LayerControl().add_to(m)

# And the Color Map legend.
colormap.add_to(m)

m
